In [14]:
%load_ext dotenv
%dotenv ../brainstation_capstone_cfg.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [15]:
import requests
import base64
import os

In [18]:
# Replace these values with your own
CLIENT_ID = os.environ["SPOTIPY_CLIENT_ID"]
CLIENT_SECRET =  os.environ["SPOTIPY_CLIENT_SECRET"]

# Encode client credentials for authorization
client_creds = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

# API endpoints
AUTH_URL = "https://accounts.spotify.com/api/token"
PLAYLISTS_URL = "https://api.spotify.com/v1/me/playlists"

# Request access token
auth_response = requests.post(
    AUTH_URL,
    data={"grant_type": "client_credentials"},
    headers={"Authorization": f"Basic {client_creds_b64}"}
)
auth_data = auth_response.json()
access_token = auth_data['access_token']
# Create a private playlist
playlist_name = "My Private Playlist"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
data = {
    "name": playlist_name,
    "public": False  # Set to False to make it private
}
response = requests.post(
    PLAYLISTS_URL,
    json=data,
    headers=headers
)

if response.status_code == 201:
    print(f"Private playlist '{playlist_name}' created successfully.")
else:
    print("Playlist creation failed.")

Playlist creation failed.


In [17]:
import requests
import base64

# Replace these values with your own
CLIENT_ID = "your_client_id"
CLIENT_SECRET = "your_client_secret"

# Encode client credentials for authorization
client_creds = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

# API endpoints
AUTH_URL = "https://accounts.spotify.com/api/token"
PLAYLISTS_URL = "https://api.spotify.com/v1/me/playlists"

# Request access token
auth_response = requests.post(
    AUTH_URL,
    data={"grant_type": "client_credentials"},
    headers={"Authorization": f"Basic {client_creds_b64}"}
)

auth_data = auth_response.json()
if "access_token" in auth_data:
    access_token = auth_data['access_token']
    print("Access token acquired successfully.")
else:
    print("Failed to acquire access token:", auth_data)

# Create a private playlist
playlist_name = "My Private Playlist"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
data = {
    "name": playlist_name,
    "public": False
}
response = requests.post(
    PLAYLISTS_URL,
    json=data,
    headers=headers
)

print("Response Status Code:", response.status_code)
print("Response JSON:", response.json())


Failed to acquire access token: {'error': 'invalid_client', 'error_description': 'Invalid client'}
Response Status Code: 401
Response JSON: {'error': {'status': 401, 'message': 'Unauthorized.'}}


In [13]:
from flask import Flask, request, redirect
import requests
import base64

app = Flask(__name__)

# Replace these values with your own
CLIENT_ID = "your_client_id"
CLIENT_SECRET = "your_client_secret"
REDIRECT_URI = "http://localhost:5000/callback"  # Update with your redirect URI
SCOPE = "playlist-modify-private"

# Step 1: Redirect the user to the authorization URL
@app.route("/login")
def login():
    auth_url = (
        f"https://accounts.spotify.com/authorize"
        f"?client_id={CLIENT_ID}"
        f"&response_type=code"
        f"&redirect_uri={REDIRECT_URI}"
        f"&scope={SCOPE}"
    )
    return redirect(auth_url)

# Step 2: Handle the authorization code and exchange for an access token
@app.route("/callback")
def callback():
    auth_code = request.args.get("code")
    
    # Exchange authorization code for access token
    auth_headers = {
        "Authorization": f"Basic {base64.b64encode((CLIENT_ID + ':' + CLIENT_SECRET).encode()).decode()}"
    }
    auth_data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": REDIRECT_URI,
    }
    
    auth_response = requests.post(
        "https://accounts.spotify.com/api/token",
        data=auth_data,
        headers=auth_headers
    )
    
    access_token = auth_response.json().get("access_token")

    # Step 3: Use the access token to create a private playlist
    playlist_headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    playlist_data = {
        "name": "My Private Playlist",
        "public": False
    }
    
    playlist_response = requests.post(
        "https://api.spotify.com/v1/me/playlists",
        json=playlist_data,
        headers=playlist_headers
    )
    
    return "Private playlist created!"

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/traitlets/config/application.py", line 991, in launch_instance
    app.initialize(argv)
  File "/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 665, in initialize
    self.init_sockets()
  File "/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 309, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "

SystemExit: 1